In [ ]:
import pandas as pd


In [ ]:
full_df = pd.read_csv('/content/output3.csv')

In [ ]:
df = full_df[['generated_summary','rating']]
df.head()

,generated_summary,rating
0,Researchers have developed an electric stimula...,2
1,Cancer researchers in Delaware believe they ha...,3
2,The effects of a probiotic supplement on mood ...,1
3,Do you know your healthy eating goals but are ...,1
4,New radioactive tracer molecules have been ide...,3


In [ ]:
import numpy as np
import torch
import random
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
seed_all(202020)

In [ ]:
def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    # if is_torch_available():
    import torch

    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # ^^ safe to call this function even if cuda is not available
    # if is_tf_available():
    import tensorflow as tf

    tf.random.set_seed(seed)
set_seed(202020)

In [ ]:
df.head()

,generated_summary,rating
0,Researchers have developed an electric stimula...,2
1,Cancer researchers in Delaware believe they ha...,3
2,The effects of a probiotic supplement on mood ...,1
3,Do you know your healthy eating goals but are ...,1
4,New radioactive tracer molecules have been ide...,3


In [ ]:
df['rating'] = df['rating'].map({0:0,1:0,2:0,3:1,4:1})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df['generated_summary'] = df['generated_summary'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_df1 = train_test_split(df, test_size=0.1,random_state=202020)

print('train shape: ',train_df.shape)
print('test shape: ',test_df1.shape)

train shape:  (270, 2)
test shape:  (30, 2)


In [ ]:
!pip install simpletransformers


# check installed version
!pip freeze | grep simpletransformers
# simpletransformers==0.28.2

     |████████████████████████████████| 225kB 5.6MB/s 
     |████████████████████████████████| 317kB 7.5MB/s 
     |████████████████████████████████| 51kB 5.8MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 
     |████████████████████████████████| 1.4MB 8.6MB/s 
     |████████████████████████████████| 2.9MB 17.1MB/s 
     |████████████████████████████████| 1.8MB 46.2MB/s 
     |████████████████████████████████| 7.4MB 49.2MB/s 
     |████████████████████████████████| 890kB 49.3MB/s 
     |████████████████████████████████| 163kB 46.7MB/s 
     |████████████████████████████████| 133kB 50.4MB/s 
     |████████████████████████████████| 102kB 11.9MB/s 
     |████████████████████████████████| 102kB 13.0MB/s 
     |████████████████████████████████| 4.5MB 42.7MB/s 
     |████████████████████████████████| 112kB 37.7MB/s 
     |████████████████████████████████| 71kB 9.5MB/s 
     |████████████████████████████████| 122kB 43.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-

simpletransformers==0.51.0


In [ ]:
from simpletransformers.classification import ClassificationModel

model = ClassificationModel("roberta", "roberta-base", use_cuda=True,num_labels=2, args={'train_batch_size':4,
                                                                          'eval_batch_size':4,
                                                                         'reprocess_input_data': True,
                                                                         'overwrite_output_dir': True,
                                                                         'fp16': False,
                                                                         'do_lower_case': False,
                                                                         'num_train_epochs': 4,
                                                                         'max_seq_length': 32,
                                                                         'regression': False,
                                                                         'manual_seed': 202020,
                                                                         "learning_rate":1e-5,
                                                                         'weight_decay':1e-5,
                                                                         "save_eval_checkpoints": False,
                                                                         "save_model_every_epoch": False,
                                                                         "silent": False})


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
# Train the model
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:387: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


(272, 0.6349840634228552)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score



def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_df1, f1=f1_multiclass, acc=accuracy_score)

result

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


{'acc': 0.6,
 'eval_loss': 0.5970257017761469,
 'f1': 0.6,
 'fn': 10,
 'fp': 2,
 'mcc': 0.2004459314343183,
 'tn': 14,
 'tp': 4}

In [ ]:
from scipy.special import softmax

raw_outputs_val = model.eval_model(test_df1)[1]
raw_outputs_val = softmax(raw_outputs_val,axis=1)[:,1]


/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:966: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


In [ ]:
outdf = pd.DataFrame(raw_outputs_val,columns=['Probabilities'])
outdf['predicted'] = np.where(outdf['Probabilities'] >0.35,1,0)
outdf['actual'] = test_df1['rating'].values

In [ ]:
from sklearn.metrics import *

In [ ]:
confusion_matrix(outdf['actual'].values,outdf['predicted'].values)

array([[13,  3],
       [ 5,  9]])

In [ ]:
f1_score(outdf['actual'].values,outdf['predicted'].values)

0.6923076923076924

In [ ]:
precision_score(outdf['actual'].values,outdf['predicted'].values)

0.75

In [ ]:
recall_score(outdf['actual'].values,outdf['predicted'].values)

0.6428571428571429

In [ ]:
outdf.head(30)

,Probabilities,predicted,actual
0,0.087920,0,0
1,0.233121,0,1
2,0.343083,0,1
3,0.477786,1,1
4,0.499985,1,0
5,0.658651,1,1
6,0.510407,1,0
7,0.186675,0,1
8,0.068350,0,0
9,0.484445,1,1
